In [1]:
# 1- IMPORTAR BIBLIOTECAS

import pandas as pd
import numpy as np
import jellyfish

In [2]:
# 2- IMPORTAR ARQUIVO

positions_user = pd.read_excel('Connections_Daniel_Para Classificar.xlsx')

C:\Users\user\mambaforge\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:


#2.3- Criar o lookup_table

# Translations between keywords and their category, as dict, as provided in question
dicpositions = {'0 - CEO, Founder, Partner': ['Fundadora', 'CEO', 'Founder', 'Co-Founder', 'President', 'Sócio', 'Cofounder', 'Vice President', 'Vice Director', 'Owner', 'Founding Member', 'Partner', 'Chairman', 'Empreendedor', 'Empreendedora', 'Entrepreneur'],
                '1 - Director of, CFO, CLO, C-Level': ['Director', 'Head', 'CFO', 'CTO', 'CLO', 'COO', 'Gerente Geral', 'Geral Manager', 'Executivo'],
                '2 - Gerente, Supervisor, Coordenador': ['Líder', 'Gestor', 'Manager', 'Administrador', 'Gerente', 'Coordenador', 'Coordenadora', 'Supervisor'],
                '3 - Engenheiro, Consultor Sênior': ['Engenheiro', 'Engineering', 'Consultor Sênior', 'Senior Consultant'],
                '4 - Consultor, Analista, Especialista': ['Design', 'Designer', 'Product', 'Consultor', 'Consultoria', 'Consultant', 'Analista', 'Analyst', 'Specialist'],
                '5 - Estagiário, Bolsista, Estudante': ['Internship','Consultor Júnior', 'Estágio', 'Iniciação Científica', 'Bolsista', 'Estagiário', 'Intern', 'Estudante'],
                '6 - Autônomo, Desempregado': ['Projetos', 'Self-Employed', 'Autônomo', 'Freelancer', 'Writer'],
                '7 - Professor, Pesquisador, Acadêmico': ['Decano', 'Schoolar', 'Acadêmico', 'Professor', 'Researcher', 'Research', 'Professora', 'Pedagógico', 'Bacharelado', 'Curso', 'MBA'],
                '8 - Advogado': ['Advogado', 'Lawyer', 'Advogada'],
                '9 - Assessor': ['Assessor', 'Assessora', 'Secretária'],
                '10 - Assistente, Auxiliar': ['Assistente', 'Auxiliar', 'Assistant'],
                'Not found': ["(blank)"]  # <-- I added this to deal with blank values
               }

# Let's expand the dict above to a DF, which makes for easier merging later
positions = []
aliases_geral = []
for key, val in dicpositions.items():
    for v in val:
        positions.append(key)
        aliases_geral.append(v)
# This will serve as our mapping table
lookup_table= pd.DataFrame({
    "alias": aliases_geral,
    "PositionsLevels": positions  
})

In [6]:
# 2.4 - PREENCHER ESPAÇOS VAZIOS E DIVIDIR CADA POSITION EM PALAVRAS

post = positions_user['Position'].fillna('(blank)')

post_df = post.str.split()


In [7]:
#3.1 - List of the alias of 1

#'pos' will always be the short from 'positions'
pos = []
n2 = len(aliases_geral)
x = 0
alias_merge = []
pos_merge = []
pos_merge_final = []


while x < len(post_df):
    pos = post_df[x]*n2
    
    aliases = []
    
    #it's to creat a variable len
    n2 = len(aliases_geral)
    i = 0
    alias = 0

    while alias < n2:
        while i < len(post_df[x]):
            aliases.append(aliases_geral[alias])    
            i += 1
        i = 0
        alias += 1       
    
    #3.2 - MAKE THE DF COMPARE
    compare = pd.DataFrame({'Alias' : aliases, 'Position_Frag' : pos})
    
    #4- ADICIONAR COLUNA COM JAIRO WINKLER
    compare['res'] = [jellyfish.jaro_winkler(x, y) for x, y in zip(compare['Position_Frag'], compare['Alias'])]
    
    #5.1- SELECIONAR O CARGO DA POSITIONS E DO ALIAS    
    cargo = []
    cargo_compare = []
    cargo.append(compare[compare['res'] == compare['res'].max()]['Position_Frag'])
    cargo_compare.append(compare[compare['res'] == compare['res'].max()]['Alias'])
    
    
    #5.2- TRANSFORMAR A SERIES EM STR
    alias_bruto = []
    pos_bruto = []
    alias_final = []
    pos_final = []
    
    alias_bruto.append(pd.Series.to_string(cargo[0]))
    pos_bruto.append(pd.Series.to_string(cargo_compare[0]))
    
    
    #6.1- SELECT ONLY THE ROLE
    alias_primario = alias_bruto[0].split()[1:]
    pos_primario = pos_bruto[0].split()[1:]
    
    
    #6.2- SELECT ONLY THE FIRST ROLE
    if len(alias_primario) > 0:
        alias_primario = alias_primario.pop(0)
        alias_final.append(alias_primario)
    
    if len(pos_primario) > 0:
        pos_primario = pos_primario.pop(0)
        pos_final.append(pos_primario)
    
    #6.3- ADD THE ROLE TO THE LIST THAT WILL MERGE WITH THE LOOKUP_TABLE
    
    alias_merge.append(alias_final[0])
    pos_merge.append(pos_final[0])

    alias = 0
    x += 1
    
##############STEPS AFTER COMPLETION OF ALIAS_MERGE#################

#7- CREATE THE DATAFRAME WITH THE ALIAS_MERGE

alias_df = pd.DataFrame({'Alias': pos_merge,                         
                         'PosMatch': alias_merge}) 

#8- MERGE WITH THE LOOKUP_TABLE
    
result_test = alias_df.merge(lookup_table, left_on="Alias", right_on="alias", how="left")

#9- ADD TO THE POSLEAN

positions_user['PoitionLevel'] = result_test['PositionsLevels']

#PREVIEW DATAFRAME - the line bellow is commented so that the data wont be exposed
#positions_user

In [8]:
# 10 - EXPORT DF

positions_user.to_excel('Conexões Daniel_Position Level_V2.xlsx', index = False)
